<h1>Introduction to AES</h1>
<p>The Advanced Encryption Standard (AES), also known by its original name <b>Rijndael</b> (Rijndael is a family of ciphers with different key and block sizes) is a specification for the encryption of electronic data established by the U.S. National Institute of Standards and Technology (NIST) in 2001 and has been adopted by the U.S. government and is now used worldwide.

AES is a subset of the Rijndael block cipher developed by two Belgian cryptographers, Vincent Rijmen and Joan Daemen, who submitted a proposal to NIST during the AES selection process. It is a symmetric-key algorithm, meaning the same key is used for both encrypting and decrypting the data.

For AES, NIST selected three members of the Rijndael family, each with a block size of 128 bits, but three different key lengths: 128, 192 and 256 bits. Over here we will be focusing on key length of 128 bits.</p>

<h2>Why AES</h2>
<p>AES supersedes the Data Encryption Standard (DES), which was published in 1977.

In 1999 the US National Institute of Standards and Technology (NIST) indicated that DES should only be used for legacy systems and instead triple DES (3DES) should be used. Even though 3DES resists brute-force attacks with today’s technology, there are several problems with it. First, it is not very efficient with regard to software implementations. DES is already not particularly well suited for software and 3DES is three times slower than DES. Another disadvantage is the relatively short block size of 64 bits, which is a drawback in certain applications</p>

<h2>Byte, Word, State</h2>

<img src="./jpeg/bbw.JPG">

<p>
<b>Byte: </b>An 8-bit hexadecimal number representing a polynomial, ranging from 0x00 till 0xff.<br>
<b>Word:  </b>A 32-bit hexadecimal number, concatenating 4 Bytes, ranging from 0x00000000 till 0xffffffff.<br>
<b>State: </b>A list consists of 4 Words, with total length of 128 bits. <br>
<b>Key: </b>A list consists of 4/6/8 Words, for 128/192/256 bits key separately.<br></p>

In [1]:
# Get the Nth Byte from the word
def GetByte(word, num):
    # Raise error if parameters out of range
    if num < 0 or num > 3:
        raise ValueError("Num out of range!")
    elif word < 0x0 or word > 0xffffffff:
        raise ValueError("Word value out of range!")
    
    # Return selected word
    else:
        return (word >> ((3-num) * 8) ) & 0xff

In [2]:
# Display a Byte in hexdecimal form
def DisplayByte(byte):
    if byte < 0x0 or byte > 0xff:
        raise ValueError("Byte value out of range!")
    else:
        print('{:02x}'.format(byte).upper(), end=' ')

# Display a Word in hexdecimal form
def DisplayWord(word):
    if word < 0x0 or word > 0xffffffff:
        raise ValueError("Word value out of range!")
    else:
        for i in range(4):
            DisplayByte(GetByte(word, i))
        print()

# Display a State in hexdecimal form
def DisplayState(state):
    for i in range(4):
        for j in range(len(state)):
            DisplayByte(GetByte(state[j], i))
        print()

<h3>Try Yourself</h3>

In [3]:
# Display a sample state
DisplayState([0x01020304, 0x11121314, 0xa1a2a3a4, 0xb1b2b3b4])

01 11 A1 B1 
02 12 A2 B2 
03 13 A3 B3 
04 14 A4 B4 


<h1>Galois Field (GF)</h1>
<p>Galois Field, named after Evariste Galois, also known as finite field, refers to a field in which there exists finitely many elements. 
    
It is particularly useful in translating computer data as they are represented in binary forms. That is, <b>computer data</b> consist of combination of two numbers, <b>0 and 1</b>, which are the components in Galois field whose number of elements is two.

Representing data as a vector in a Galois Field allows mathematical operations to scramble data easily and effectively.</p>

<h1>Galois field GF(2<sup>m</sup>) in AES</h1>

<h2>Finite Field Arithmetic in GF</h2>

<p>Unlike working in the Euclidean space, addition (and subtraction) and multiplication in Galois Field requires <b>additional steps</b>.
    
The elements of a finite field can be represented in several different ways. In AES the finite field contains 256 elements and is denoted as GF(2<sup>8</sup>).This field was chosen because each of the field elements can be represented by one byte. For the <b>S-Box and MixColumn transforms</b>, AES treats every byte of the internal data path as an element of the field GF(2<sup>8</sup>) and manipulates the data by performing arithmetic in this finite field. 
    
However, if the order of a finite field is not prime, and 2<sup>8</sup> is clearly not a prime, the addition and multiplication operation cannot be represented by addition and multiplication of integers modulo 2<sup>8</sup>. Such fields with m> 1 are called extension fields. Elements of extension fields can be represented as polynomials, and that computation in the extension field is achieved by performing a certain type of polynomial arithmetic.
    
The polynomials have a maximum degree of m−1, so that there are m coefficients in total for every element. In the field GF(2<sup>8</sup>), which is used in AES. A byte b, consisting of bits b<sub>7</sub> b<sub>6</sub> b<sub>5</sub> b<sub>4</sub> b<sub>3</sub> b<sub>2</sub> b<sub>1</sub> b<sub>0</sub>, is considered as a polynomial with coefficient in {0,1}:<br>\begin{align} b_{7}x^{7} + b_{6}x^{6} + b_{5}x^{5} + b_{4}x^{4} + b_{3}x^{3} + b_{2}x^{2} + b_{1}x + b_{0} \end{align}</p>

<p><b>Example:</b> the byte with hexadecimal value ‘0x57’ (binary <b>0101 0111</b>) corresponds with polynomial: <br>\begin{align} x^{6} + x^{4} + x^{2} + x + 1 \end{align}</p>

<h2>Addition and Subtraction in GF(2<sup>8</sup>)</h2>
<p>The key addition layer of AES uses addition.They are simply achieved by performing standard polynomial addition and subtraction: We merely add or subtract coefficients with equal powers of x. The calculation is the same as addition modulo 2 which is equal to bitwise XOR. Let’s have a look at an example in the field GF(2<sup>8</sup>) which is used in AES:

Here is how the sumC(x)=A(x)+B(x) of two elements from GF(2<sup>8</sup>) is computed:
</p>
<img src="./jpeg/additionGF.JPG">

<p>Note that if we computed the difference of the two polynomials A(x)−B(x) from the example above, we would get the same result as for the sum.</p>

<h2>Multiplication in GF(2<sup>8</sup>)</h2>

<p>Multiplication in Galois Field, however, requires more tedious work. Multiplication in GF(2<sup>8</sup>) is the core operation of the <b>MixColumn transformation</b> of AES. 
    
In a first step, two elements (represented by their polynomials) of a finite field GF(2<sup>m</sup>) are multiplied using the standard polynomial multiplication rule: The product of the multiplication is divided by a certain polynomial, and we consider only the remainder after the polynomial division. We need <b>irreducible polynomials</b> for the module reduction.

In AES, irreducible polynomial = x<sup>8</sup> + x<sup>4</sup> + x<sup>3</sup> + x<sup>1</sup> + 1 is used. It is part of the AES specification. which is equal to HEX = 0x11b or BIN = 100011011 

<b>Example</b>

<p>we take the irreducible polynomial modulo P(x) as x<sup>8</sup> + x<sup>4</sup> + x<sup>3</sup> + x<sup>1</sup> + 1. To calculate 84 · 13, we need to go through several steps. First, we compute the product of the polynomial

<br>\begin{align} 84 * 13 = (x^{6} + x^{4} + x^{2}) * (x^{3} + x^{2} + 1)(mod  P(x))\end{align}
<br>\begin{align} = (x^{9} + x^{8} + x^{7} + 2*x^{6} + x^{5} + 2*x^{4} +x^{2})(mod  P(x))\end{align}
<br>\begin{align} = (x^{9} + x^{8} + x^{7} + x^{5} +x^{2})(mod  P(x))\end{align}


Then we use long division to compute the reduced polynomial as follows.

<br>\begin{align}(x^{9} + x^{8} + x^{7} + x^{5} +x^{2})/(x^{8} + x^{4} + x^{3} + x^{1} + 1)\end{align}

<br>\begin{align} = (x^{1} + 1) R (1)\end{align}

The remainder is what we're looking for, so the product of 84 · 13 is equal to 1. Also that 84 and 13 are multiplicative inverse pairs.

</p>

In [4]:
# Code reference: Wikipedia - Finite field arithmetic
# https://en.wikipedia.org/wiki/Finite_field_arithmetic#Rijndael.27s_finite_field
# Multiply two numbers in the GF(2^8) finite field defined
# using the Russian Peasant Multiplication algorithm
def PolyMul(byte_a, byte_b, irr_poly = 0x11b):
    product = 0
    while byte_a and byte_b:
        
        # if b is odd, then add the corresponding a to p 
        # (final product = sum of all a's corresponding to odd b's)
        if byte_b & 0x01:
            # since we're in GF(2^8), addition is an XOR
            product ^= byte_a

        # GF modulo: if a >= 128, then it will overflow when shifted left, so reduce
        if byte_a & 0x80:
            # XOR with the irreducible polynomial
            byte_a = (byte_a << 1) ^ irr_poly
            
        else:
            # equivalent to a*2
            byte_a <<= 1

        # equivalent to b/2
        byte_b >>= 1

    return product

<h3>Try Yourself</h3>

In [5]:
# 0x3d multiply 0x2a under default irreducible polynomial 0x11b
DisplayByte(PolyMul(0x3d, 0x2a))

68 

<h2>Inversion in GF(2<sup>m</sup>)</h2>
<p>
    
Inversion in GF(2<sup>8</sup>) is the core operation of the <b>Byte Substitution transformation</b>,
which contains the AES S-Boxes. For a given finite field GF(2<sup>m</sup>) and the corresponding
irreducible reduction polynomial P(x), the inverse A−1 of a nonzero element
A ∈ GF(2<sup>m</sup>) is defined as:

<img src="./jpeg/inverse.JPG">

For small fields — in practice this often means fields with 216 or fewer elements
— lookup tables which contain the precomputed inverses of all field elements are
often used. Table below shows the example of s-box.

<img src="./jpeg/sbox.JPG">

The table contains all inverses in GF(2<sup>8</sup>) modulo P(x) = x<sup>8</sup> + x<sup>4</sup> + x<sup>3</sup> + x<sup>1</sup> + 1 in
hexadecimal notation. A special case is the entry for the field element 0, for which
an inverse does not exist. However, for the AES S-Box, a substitution table is needed
that is defined for every possible input value.</p>

In [6]:
# Find the inverse of byte by GF(2^8) modulo irr_poly by brute-force search
def FindInverse(byte, irr_poly):
    if byte == 0:
        return 0

    else:
        # Multiplying byte by every number in GF(2^8) until the product is 1
        # At most 128 loops per execution
        for i in range(2**8):
            if PolyMul(byte, i, irr_poly) == 1:
                return i
        
        # If no inverse found
        # Throw error here
        raise IndexError("No inverse found!")

<h3>Try Yourself</h3>

In [7]:
# Inverse of 0x25 under default irreducible polynomial 0x11b
DisplayByte(FindInverse(0x25, 0x11b))

4D 

In [8]:
# Find the result of a polynomial after left circular shift by n bits
def LftSftByte(byte, bit):
    return ((byte << bit % 8) & (2**8 - 1)) | (byte >> (8 - bit % 8))

<h3>Try Yourself</h3>

In [9]:
# left circular shift 3 bits for polynomial 0x23
DisplayByte(LftSftByte(0x23, 3))

19 

<h1>AES Look up table (S-Box)</h1>
<p>
The Rijndael S-box is a substitution box (lookup table) used in the Rijndael cipher, which the Advanced Encryption Standard (AES) cryptographic algorithm was based on. The S-box maps an 8-bit input to an 8-bit output. Both the input and output are interpreted as polynomials over GF(2). First, the input is mapped to its multiplicative inverse in GF(2<sup>8</sup>) = GF(2)[x]/(x<sup>8</sup> + x<sup>4</sup> + x<sup>3</sup> + x + 1). The multiplicative inverse is then transformed using the following affine transformation:</p>

<img src="./jpeg/sboxmaking.JPG">

<p>where [s<sub>7</sub>, …, s<sub>0</sub>] is the S-box output and [b<sub>7</sub>, …, b<sub>0</sub>] is the multiplicative inverse as a vector.

This affine transformation is the sum of multiple rotations of the byte as a vector, where addition is the XOR operation:
<img src="./jpeg/sboxop.JPG">

where b represents the multiplicative inverse, (+) is the bitwise XOR operator, <<< is a left bitwise circular shift, and the constant 63<sub>16</sub> = 01100011<sub>2</sub> is given in hexadecimal.

Example of shifting bits:

d2<sub>16</sub> = 1101 0010<sub>2</sub>

and we shift the bit to the left by 1:

1101 0010 shift left by 1 > 1010 0101<sub>2</sub> = a5<sub>16</sub></p>

In [10]:
# Initiate an 16x16 look up table and return the result in array form
def SBoxInit(irr_poly=0x11b, add_poly=0x63):
    # Initialize look up table by filling -1 in a 1-Dimensional list with size of 256
    look_up_table = [-1 for x in range(256)]
    
    # Filling the look up table cell by cell by a for loop
    for x in range(256):
        inv_poly = FindInverse(x, irr_poly)
        look_up_table[x] = inv_poly ^ LftSftByte(inv_poly, 1) ^ LftSftByte(inv_poly, 2) \
            ^ LftSftByte(inv_poly, 3) ^ LftSftByte(inv_poly, 4) ^ add_poly

    return look_up_table

In [11]:
# Create a constant sbox and use defined function to initialize
S_BOX = SBoxInit()

In [12]:
# Print the resulting sbox / rsbox as a table
def DisplayLUT(look_up_table):
    # Print header of the sbox / rsbox
    print("x\\y", end="")
    for num in range(16):
        print(" {:1x}".format(num).upper(), end=" ")
    print()

    # Print values within sbox / rsbox
    for i in range(16):
        print("{:1x} ".format(i).upper(), end=" ")
        for j in range(16):
            print("{:02x}".format(look_up_table[16*i + j]).upper(), end=" ")
        print()

In [13]:
# Display generated s-box value by the funtion above
DisplayLUT(S_BOX)

x\y 0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F 
0  63 7C 77 7B F2 6B 6F C5 30 01 67 2B FE D7 AB 76 
1  CA 82 C9 7D FA 59 47 F0 AD D4 A2 AF 9C A4 72 C0 
2  B7 FD 93 26 36 3F F7 CC 34 A5 E5 F1 71 D8 31 15 
3  04 C7 23 C3 18 96 05 9A 07 12 80 E2 EB 27 B2 75 
4  09 83 2C 1A 1B 6E 5A A0 52 3B D6 B3 29 E3 2F 84 
5  53 D1 00 ED 20 FC B1 5B 6A CB BE 39 4A 4C 58 CF 
6  D0 EF AA FB 43 4D 33 85 45 F9 02 7F 50 3C 9F A8 
7  51 A3 40 8F 92 9D 38 F5 BC B6 DA 21 10 FF F3 D2 
8  CD 0C 13 EC 5F 97 44 17 C4 A7 7E 3D 64 5D 19 73 
9  60 81 4F DC 22 2A 90 88 46 EE B8 14 DE 5E 0B DB 
A  E0 32 3A 0A 49 06 24 5C C2 D3 AC 62 91 95 E4 79 
B  E7 C8 37 6D 8D D5 4E A9 6C 56 F4 EA 65 7A AE 08 
C  BA 78 25 2E 1C A6 B4 C6 E8 DD 74 1F 4B BD 8B 8A 
D  70 3E B5 66 48 03 F6 0E 61 35 57 B9 86 C1 1D 9E 
E  E1 F8 98 11 69 D9 8E 94 9B 1E 87 E9 CE 55 28 DF 
F  8C A1 89 0D BF E6 42 68 41 99 2D 0F B0 54 BB 16 


<h2>Inverse S-box</h2>
<p>The inverse S-box is simply the S-box run in reverse. For example, the inverse S-box of b8<sub>16</sub> is 9a<sub>16</sub>. It is calculated by first calculating the inverse affine transformation of the input value, followed by the multiplicative inverse. The inverse affine transformation is as follows:

<img src="./jpeg/invsboxmaking.JPG">


The inverse affine transformation also represents the sum of multiple rotations of the byte as a Vector, where addition is the XOR operation:

<img src="./jpeg/invsboxop.JPG">

where (+) is the bitwise XOR operator, <<< is a left bitwise circular shift, and the constant 5<sub>16</sub> = 00000101<sub>2</sub> is given in hexadecimal.
</p>

In [14]:
def RSBoxInit():
    # Initialize look up table by filling -1 in a 1-Dimensional list with size of 256
    look_up_table = [-1 for x in range(256)]
    
    # Fill in inverse of look up table
    for x in range(256):
        look_up_table[x] = S_BOX.index(x)
    
    return look_up_table

In [15]:
# Create a constant rsbox and use defined function to initialize
RS_BOX = RSBoxInit()

In [16]:
# Display generated reverse s-box value by the funtion above
DisplayLUT(RS_BOX)

# Below is the generated reverse S-box

x\y 0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F 
0  52 09 6A D5 30 36 A5 38 BF 40 A3 9E 81 F3 D7 FB 
1  7C E3 39 82 9B 2F FF 87 34 8E 43 44 C4 DE E9 CB 
2  54 7B 94 32 A6 C2 23 3D EE 4C 95 0B 42 FA C3 4E 
3  08 2E A1 66 28 D9 24 B2 76 5B A2 49 6D 8B D1 25 
4  72 F8 F6 64 86 68 98 16 D4 A4 5C CC 5D 65 B6 92 
5  6C 70 48 50 FD ED B9 DA 5E 15 46 57 A7 8D 9D 84 
6  90 D8 AB 00 8C BC D3 0A F7 E4 58 05 B8 B3 45 06 
7  D0 2C 1E 8F CA 3F 0F 02 C1 AF BD 03 01 13 8A 6B 
8  3A 91 11 41 4F 67 DC EA 97 F2 CF CE F0 B4 E6 73 
9  96 AC 74 22 E7 AD 35 85 E2 F9 37 E8 1C 75 DF 6E 
A  47 F1 1A 71 1D 29 C5 89 6F B7 62 0E AA 18 BE 1B 
B  FC 56 3E 4B C6 D2 79 20 9A DB C0 FE 78 CD 5A F4 
C  1F DD A8 33 88 07 C7 31 B1 12 10 59 27 80 EC 5F 
D  60 51 7F A9 19 B5 4A 0D 2D E5 7A 9F 93 C9 9C EF 
E  A0 E0 3B 4D AE 2A F5 B0 C8 EB BB 3C 83 53 99 61 
F  17 2B 04 7E BA 77 D6 26 E1 69 14 63 55 21 0C 7D 


In [17]:
def GetSBoxValue(byte):
    # byte should be ranging from 0x00 till 0xff
    # Throw error if out of range
    if byte < 0 or byte > 0xff :
        # Throw error here
        raise ValueError("Input out of range!")
    
    # Return sbox value
    else:
        return S_BOX[byte]

In [18]:
def GetRSBoxValue(byte):
    # byte should be ranging from 0x00 till 0xff
    # Throw error if out of range
    if byte < 0 or byte > 0xff :
        # Throw error here
        raise ValueError("Input out of range!")
    
    # Return rsbox value
    else:
        return RS_BOX[byte]

<h1>AES Encryption Algorithm</h1>
<p>The encryption phase of AES can be broken into three phases: the initial round, the main rounds, and the final round. All of the phases use the same sub-operations in different combinations as follows:

<b>KeyExpansion</b>—round keys are derived from the cipher key using Rijndael's key schedule. AES requires a separate 128-bit round key block for each round plus one more.

<h3>Initial round key addition:</h3>

<b>AddRoundKey</b>—each byte of the state is combined with a byte of the round key using bitwise xor. As mentioned previously, three key lengths must be supported by Rijndael as this was an NIST design requirement. The number of internal rounds to the number of key length are : <b>(128 - 10 rounds ,192 - 12 rounds ,256 - 14 rounds)</b><br>

<h3>Main Rounds (9, 11 or 13 rounds based on different key length used):</h3>

<b>SubBytes</b>—a non-linear substitution step where each byte is replaced with another according to a lookup table.<br>

<b>ShiftRows</b>—a transposition step where the last three rows of the state are shifted cyclically a certain number of steps.<br>

<b>MixColumns</b>—a linear mixing operation which operates on the columns of the state, combining the four bytes in each column.<br>

<b>AddRoundKey</b>—each byte of the state is combined with a byte of the round key using bitwise xor. As mentioned previously, three key lengths must be supported by Rijndael as this was an NIST design requirement. The number of internal rounds to the number of key length are : <b>(128 - 10 rounds ,192 - 12 rounds ,256 - 14 rounds)</b><br>

<h3>Final round (making 10, 12 or 14 rounds in total):</h3>

<b>SubBytes</b>—a non-linear substitution step where each byte is replaced with another according to a lookup table.<br>

<b>ShiftRows</b>—a transposition step where the last three rows of the state are shifted cyclically a certain number of steps.<br>

<b>AddRoundKey</b>—a linear mixing operation which operates on the columns of the state, combining the four bytes in each column.<br></p>

<h2>AES Key Schedule</h2>
<p>This section describes how round keys used in AES being generated.

Key Addition layer A 128-bit round key, or subkey, which has been derived from the main key in the key schedule, is XORed to the state.

The key schedule takes the original input key (of length 128, 192 or 256 bit) and derives the subkeys used in AES. Note that an XOR addition of a subkey is used both at the input and output of AES. This process is sometimes referred to as key whitening. The number of subkeys is equal to the number of rounds plus one, due to the key needed for key whitening in the first key addition layer, cf. Fig. 4.2. Thus, for the key length of 128 bits, the number of rounds is 10, and there are 11 subkeys, each of 128 bits.</p>

<h3>Round Constant</h3>
<p>In the AES algorithm, keys are expanded by something called <b>round constant or key scedule round</b>. The output of this is known as round key. Using XOR encryption, or XOR cipher, the output of each 10 rounds of original plain text is used to make the next input.
    
The <b>round constant</b> can be obtain easily by multiplying the first round [01 00 00 00] with the second round [02 00 00 00] to get the third row which is [04 00 00 00]. And continue multiplying the second with the third to get the forth round contant etc.

In [19]:
# Use code below to generate first 10 rcon values because AES only uses up to 10 rcon value:
def RconInit():
    # Initialize a 1-dimentional array with value -1
    rcon_table = [0 for x in range(11)]
    
    for x in range(11):
        if x == 0 or x == 1:
            rcon_table[x] = x
        elif x  > 1 and rcon_table[x - 1] < 0x80: # 0x80 = 128
            rcon_table[x] = 2 * rcon_table[x-1]
        else:
            rcon_table[x] = (2 * rcon_table[x-1]) ^ 0x1b
            
        # To avoid overflow
        if rcon_table[x] > 255:
            rcon_table[x] -= 256
    
    return rcon_table

In [20]:
# Initialize Constant rcon
RCON_TABLE = RconInit()

In [21]:
# Function to display first 10 values of rcon table
def DisplayRcon(array):
    for num in range(1,11):
        print("{:2}".format(num), end=" ")
    print()
        
    for i in range(1, 11):
        print("{:02x}".format(array[i]).upper(), end=" ")

In [22]:
DisplayRcon(RCON_TABLE)

 1  2  3  4  5  6  7  8  9 10 
01 02 04 08 10 20 40 80 1B 36 

In [23]:
def GetRconValue(round):
    return RCON_TABLE[round]

<h3>Try Yourself</h3>

In [24]:
# Display 9th item in rcon table
DisplayByte(GetRconValue(9))

1B 

<h2>Key Expandsion</h2>
<p>After getting the round constant, we can start expanding our key for the AddRoundKey process.</p>

<img src="./gif/rot_word.gif">

In [25]:
# Find the result of a 32-bit word after left circular shift by n bits
def LftSftWord(hex_word, bit):
    if hex_word < 0x0 or hex_word > 0xffffffff:
        raise ValueError("Input word out of range!")
    else:
        return ((hex_word << bit % 32) & (2**32 - 1)) | (hex_word >> (32 - bit % 32))

<h3>Try Yourself</h3>

In [26]:
# Left Circular Shift 8 bits
DisplayWord(LftSftWord(0x09cf4f3c, 8))

CF 4F 3C 09 


<img src="./gif/subbyteround.gif">

In [27]:
def SubWord(word):
    # Result_List
    result = 0
    
    # Literate through 4 Bytes
    for i in range(4):
        tmp = GetByte(word, i)
            
        tmp = GetSBoxValue(tmp)
            
        # Append the result to the end
        result = result << 8 | tmp
        
    return result

<img src="./gif/additionround.gif">

In [28]:
def ExpandKey(key):
    # Expands an 128,192,256 bytes key into an 1408,2496,3840 bytes key
    # Key is an array with 32-bit words.
    # AES-128: 4 words, AES-192: 6 words, AES-256: 8 words
    N = len(key)
    R = N + 7
    result = [0 for x in range(4 * R)]
    
    for i in range(4 * R):
        if i < N:
            result[i] = key[i]
            
        elif i >= N and i % N == 0:
            result[i] = result[i - N] ^ \
            SubWord(LftSftWord(result[i - 1], 8)) ^ \
            (GetRconValue(int(i / N)) << 24)
        
        elif i >= N and N > 6 and i % N == 4:
            result[i] = result[i - N] ^ SubWord(result[i - 1])
            
        else:
            result[i] = result[i - N] ^ result[i - 1]
    
    return result

In [29]:
# Get round key from the expanded key
def GetRoundKey(expandedkey, round):
    if round <= 0 or round > len(expandedkey) / 4:
        return -1
    else:
        return expandedkey[((round)*4):((round+1)*4)]

<h3>Try Yourself</h3>

In [30]:
expandedkey = ExpandKey([0x2b7e1516, 0x28aed2a6, 0xabf71588, 0x09cf4f3c])

In [31]:
# Get the 3rd round key from the expanded key
DisplayState(GetRoundKey(expandedkey, 3))

3D 47 1E 6D 
80 16 23 7A 
47 FE 7E 88 
7D 3E 44 3B 


<h2>SubBytes</h2>

<p><b>SubBytes</b> or <b>Byte Substitution Layer</b> is a non-linear substitution step where each byte is replaced with another according to a lookup table (S-Box). The Byte Substitution layer can be viewed as a row of 16 parallel S-Boxes, each with 8 input and output bits. 
    
The S-Box substitution is a bijective mapping, which mean each of the 2<sup>8</sup> = 256 possible input elements is one-to-one mapped to one output element. This allows us to <b>uniquely reverse the S-Box</b>, which is needed for decryption. In software implementations the S-Box is usually realized as a 256-by-8 bit lookup table with fixed entries.</p>

<img src="./gif/sub_bytes.gif">

In [32]:
def SubBytes(state):
    # State is a 4-word list
    result = [0 for x in range(len(state))]
    
    # Literate through 4 words
    for i in range(4):
        # Literate through 4 bytes
        for j in range(4):
            byte = GetByte(state[i], j)
            
            # Substitute byte with previous generated S-Box
            byte = GetSBoxValue(byte)
            
            # Append the result to the end
            result[i] = result[i] << 8 | byte
            
    return result

In [33]:
# Recall the S-box generated before
DisplayLUT(S_BOX)

# Below is the correspoding s-box use for the SubByte step

x\y 0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F 
0  63 7C 77 7B F2 6B 6F C5 30 01 67 2B FE D7 AB 76 
1  CA 82 C9 7D FA 59 47 F0 AD D4 A2 AF 9C A4 72 C0 
2  B7 FD 93 26 36 3F F7 CC 34 A5 E5 F1 71 D8 31 15 
3  04 C7 23 C3 18 96 05 9A 07 12 80 E2 EB 27 B2 75 
4  09 83 2C 1A 1B 6E 5A A0 52 3B D6 B3 29 E3 2F 84 
5  53 D1 00 ED 20 FC B1 5B 6A CB BE 39 4A 4C 58 CF 
6  D0 EF AA FB 43 4D 33 85 45 F9 02 7F 50 3C 9F A8 
7  51 A3 40 8F 92 9D 38 F5 BC B6 DA 21 10 FF F3 D2 
8  CD 0C 13 EC 5F 97 44 17 C4 A7 7E 3D 64 5D 19 73 
9  60 81 4F DC 22 2A 90 88 46 EE B8 14 DE 5E 0B DB 
A  E0 32 3A 0A 49 06 24 5C C2 D3 AC 62 91 95 E4 79 
B  E7 C8 37 6D 8D D5 4E A9 6C 56 F4 EA 65 7A AE 08 
C  BA 78 25 2E 1C A6 B4 C6 E8 DD 74 1F 4B BD 8B 8A 
D  70 3E B5 66 48 03 F6 0E 61 35 57 B9 86 C1 1D 9E 
E  E1 F8 98 11 69 D9 8E 94 9B 1E 87 E9 CE 55 28 DF 
F  8C A1 89 0D BF E6 42 68 41 99 2D 0F B0 54 BB 16 


<h3>Try Yourself</h3>

In [34]:
DisplayState(SubBytes([0x193de3be, 0xa0f4e22b, 0x9ac68d2a, 0xe9f84808]))

D4 E0 B8 1E 
27 BF B4 41 
11 98 5D 52 
AE F1 E5 30 


<h1>Diffusion Layer</h1>


<p>In AES, the Diffusion layer consists of two sublayers, the <b>ShiftRows</b> transformation and the <b>MixColumn</b> transformation. Diffusion is the spreading of the influence of individual bits over the entire state. Unlike the nonlinear S-Box, the diffusion layer performs a linear operation on state matrices.</p>

<h2>ShiftRows</h2>


<p>The ShiftRows transformation cyclically shifts the second row of the state matrix by three bytes to the right, the third row by two bytes to the right and the fourth row by one byte to the right. The first row is not changed by the ShiftRows transformation. The purpose of the ShiftRows transformation is to increase the diffusion properties of AES. If the input of the ShiftRows sublayer is given as a state matrix:</p>

<img src="./gif/shift_rows.gif">

In [35]:
def ShiftRows(state):
    # State is a 4-word list
    result = [0 for x in range(len(state))]
    
    # Shift-row operation
    for i in range(len(state)):
        result[i] = (state[i] & 0xff000000) | (state[i - 3] & 0xff0000)\
        | (state[i - 2] & 0xff00) | (state[i - 1] & 0xff)
    
    return result

<h3>Try Yourself</h3>

In [36]:
DisplayState(ShiftRows([0xd42711ae, 0xe0bf98f1, 0xb8b45de5, 0x1e415230]))

D4 E0 B8 1E 
BF B4 41 27 
5D 52 11 98 
30 AE F1 E5 


<h2>MixColumns</h2>

<p>The MixColumns step is a linear transformation which mixes each column of the state matrix. Since every input byte influences four output bytes, the MixColumn operation is the <b>major diffusion element</b> in AES. The combination of the ShiftRows and MixColumn layer makes it possible that after only three rounds every byte of the state matrix depends on all 16 plaintext bytes.

Each 4-byte column is considered as a vector and multiplied by a fixed
4×4 matrix. The matrix contains constant entries. Multiplication and addition of the coefficients is done in GF(2<sup>8</sup>). As an example, we show how the first four output bytes are computed:</p>

<img src="./gif/mix_columns.gif">

In [37]:
# Constant matrix to be used in MixColumn operation
MIXCOL_MATRIX = [
    [0x2, 0x3, 0x1, 0x1],
    [0x1, 0x2, 0x3, 0x1],
    [0x1, 0x1, 0x2, 0x3],
    [0x3, 0x1, 0x1, 0x2]
]

In [38]:
def MixColumns(state):
    result = [0 for x in range(len(state))]
    
    # Matrix multiplication
    # Of the state with the matrix constant
    for i in range (4):
        for j in range(4):
            # Initialize tmp
            tmp = 0
            
            for k in range(4):
                tmp ^= PolyMul(GetByte(state[i],k), MIXCOL_MATRIX[j][k])
                
            result[i] = (result[i] << 8) | tmp

    return result

<h3>Try Yourself</h3>

In [39]:
DisplayState(MixColumns([0xd4bf5d30, 0xe0b452ae, 0xb84111f1, 0x1e2798e5]))

04 E0 48 28 
66 CB F8 06 
81 19 D3 26 
E5 9A 7A 4C 


<h1>AddRoundKey</h1>

<p>AddRoundKey or called as <b>Key Additional layer</b> are step where addition of both current 16-byte state matrix and a subkey which also consists of 16 bytes (128 bits) through a bitwise XOR operation. Note that the XOR operation is equal to addition in the Galois field GF(2). </p>


<img src="./gif/add_round_key.gif">

In [40]:
def AddRoundKey(state, key):
    # State is a 4-word list
    result = [0 for x in range(len(state))]
    
    for i in range (len(state)):
        result[i] = state[i] ^ key[i]
        
    return result

<h3>Try Yourself</h3>

In [41]:
DisplayState(AddRoundKey([0x046681e5, 0xe0cb199a, 0x48f8d37a, 0x2806264c],
           [0xa0fafe17, 0x88542cb1, 0x23a33939, 0x2a6c7605]))

A4 68 6B 02 
9C 9F 5B 6A 
7F 35 EA 50 
F2 2B 43 49 


<h1>Combine All AES encryption function</h1>

In [42]:
def Round(state, roundkey):
    result = SubBytes(state)
    result = ShiftRows(result)
    result = MixColumns(result)
    result = AddRoundKey(result, roundkey)
    
    return result

In [43]:
def FinalRound(state, roundkey):
    result = SubBytes(state)
    result = ShiftRows(result)
    result = AddRoundKey(result, roundkey)
    
    return result

In [44]:
def AESEncrypt(state, cipherkey):
    # To expand the cipher key
    expandedkey = ExpandKey(cipherkey)
    
    # Add cipher key once beforehead
    result = AddRoundKey(state, cipherkey)
    
    # Determine number of rounds based on cipher key length
    if len(cipherkey) == 4: # 128-bits key, 10 rounds
        roundnum = 10
    elif len(cipherkey) == 6: #192-bits key, 12 rounds
        roundnum = 12
    elif len(cipherkey) == 8: #256-bits key, 14 rounds
        roundnum = 14
    else:
        # Throw error here
        raise ValueError("Cipher key format incorrect!")
    
    # Loop through Nr-1 rounds
    for i in range(1, roundnum):
        result = Round(result, GetRoundKey(expandedkey, i))
    
    # Final Round
    result = FinalRound(result, GetRoundKey(expandedkey, roundnum))
    
    return result

<h3>Try Yourself</h3>

In [45]:
tstate = [0x3243f6a8, 0x885a308d, 0x313198a2, 0xe0370734]
tkey = [0x2b7e1516, 0x28aed2a6, 0xabf71588, 0x09cf4f3c]

tresult = AESEncrypt(tstate, tkey)

DisplayState(tresult)

39 02 DC 19 
25 DC 11 6A 
84 09 85 0B 
1D FB 97 32 


<h1>AES Decryption Algorithm</h1>
<p>This section describes AES Decryption Algorithm. Because AES is not based on a Feistel network, all layers must actually be inverted, for example:

<b>SubBytes layer</b> --> <b>Invert Byte Substitution layer</b>

<b>ShiftRows layer</b> --> <b>Invert ShiftRows layer</b>

<b>MixColumns layer</b> --> <b>Invert MixColumn layer</b>. 

There is no Invert AddRoundKey layer since AddRoundKey layer is just the inverse to itself.
    
However, as we will see, it turns out that the inverse layer operations are fairly similar to the layer operations used for encryption. In addition, the order of the subkeys is reversed, So we need a reversed key schedule.</p>

<h2>Invert SubByte</h2>
<p>The inverse S-Box is used when decrypting a ciphertext. Since the AES S-Box is a bijective which is a one-to-one mapping, it is possible to construct an inverse S-Box.</p>

In [46]:
# Recall the reverse S-box generated by the function before
DisplayLUT(RS_BOX)

# Below is the generated reverse S-box

x\y 0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F 
0  52 09 6A D5 30 36 A5 38 BF 40 A3 9E 81 F3 D7 FB 
1  7C E3 39 82 9B 2F FF 87 34 8E 43 44 C4 DE E9 CB 
2  54 7B 94 32 A6 C2 23 3D EE 4C 95 0B 42 FA C3 4E 
3  08 2E A1 66 28 D9 24 B2 76 5B A2 49 6D 8B D1 25 
4  72 F8 F6 64 86 68 98 16 D4 A4 5C CC 5D 65 B6 92 
5  6C 70 48 50 FD ED B9 DA 5E 15 46 57 A7 8D 9D 84 
6  90 D8 AB 00 8C BC D3 0A F7 E4 58 05 B8 B3 45 06 
7  D0 2C 1E 8F CA 3F 0F 02 C1 AF BD 03 01 13 8A 6B 
8  3A 91 11 41 4F 67 DC EA 97 F2 CF CE F0 B4 E6 73 
9  96 AC 74 22 E7 AD 35 85 E2 F9 37 E8 1C 75 DF 6E 
A  47 F1 1A 71 1D 29 C5 89 6F B7 62 0E AA 18 BE 1B 
B  FC 56 3E 4B C6 D2 79 20 9A DB C0 FE 78 CD 5A F4 
C  1F DD A8 33 88 07 C7 31 B1 12 10 59 27 80 EC 5F 
D  60 51 7F A9 19 B5 4A 0D 2D E5 7A 9F 93 C9 9C EF 
E  A0 E0 3B 4D AE 2A F5 B0 C8 EB BB 3C 83 53 99 61 
F  17 2B 04 7E BA 77 D6 26 E1 69 14 63 55 21 0C 7D 


In [47]:
def InvSubBytes(state):
    # State is a 4-word list
    result = [0 for x in range(len(state))]
    
    # Literate through 4 words
    for i in range(4):
        # Literate through 4 Bytes
        for j in range(4):
            tmp = GetByte(state[i], j)
            
            tmp = GetRSBoxValue(tmp)
            
            # Append the result to the end
            result[i] = result[i] << 8 | tmp
            
    return result

<h3>Try Yourself</h3>

In [48]:
#Initial data state
stateInvSub = [0xd42711ae, 0xe0bf98f1, 0xb8b45de5, 0x1e415230]

#Data going through SubByte Process
stateInvSub = SubBytes(stateInvSub)

DisplayState(stateInvSub)
    
#Output after SubByte Process

48 E1 6C 72 
CC 08 8D 83 
82 46 4C 00 
E4 A1 D9 04 


In [49]:
#Data going trough Inverse SubByte Process
stateInvSub = InvSubBytes(stateInvSub)

DisplayState(stateInvSub)
    
#Output after Reverse SubByte Process

D4 E0 B8 1E 
27 BF B4 41 
11 98 5D 52 
AE F1 E5 30 


<h2>Inverse ShiftRows</h2>
<p>In order to reverse the ShiftRows operation of the encryption algorithm, we must shift the rows of the state matrix in the opposite direction. The first row is not changed by the inverse ShiftRows transformation. </p>

In [50]:
def InvShiftRows(state):
    
    # State is a 4-word list
    result = [0 for x in range(len(state))]
    
    # Shift-row operation
    # On Encryption, we shifted second row to the left by 1
    # On Decryption, we just do the opposite by shifting it to the right by 1
    # Vice versa for third and forth row, first row is untouch.
    for i in range(len(state)):
        result[i] = (state[i] & 0xff000000) | (state[i - 1] & 0xff0000)\
        | (state[i - 2] & 0xff00) | (state[i - 3] & 0xff)
    
    return result

<h3>Try Yourself</h3>

In [51]:
#Initial Data State
StateInvShiftRow = [0xd42711ae, 0xe0bf98f1, 0xb8b45de5, 0x1e415230]

#Data going through ShiftRow Process
StateInvShiftRow = ShiftRows(StateInvShiftRow)

DisplayState(StateInvShiftRow)
#Output data after ShiftRow Process

D4 E0 B8 1E 
BF B4 41 27 
5D 52 11 98 
30 AE F1 E5 


In [52]:
#Data going through InvShiftRow Process
StateInvShiftRow = InvShiftRows(StateInvShiftRow)

DisplayState(StateInvShiftRow)
#Output data after InvShiftRow Process

D4 E0 B8 1E 
27 BF B4 41 
11 98 5D 52 
AE F1 E5 30 


<h2>Inverse MixColumns</h2>

<p>In order to reverse the MixColumn operation, the inverse of its matrix must be used. The input is a 4-byte column of the INV_MIXCOL_MATRIX which is multiplied by the inverse 4×4 matrix. The matrix contains constant entries. Multiplication and addition of the coefficients is done in GF(2<sup>8</sup>).

In [53]:
# Constant matrix to be used in MixColumn operation
INV_MIXCOL_MATRIX = [
    [0xe, 0xb, 0xd, 0x9],
    [0x9, 0xe, 0xb, 0xd],
    [0xd, 0x9, 0xe, 0xb],
    [0xb, 0xd, 0x9, 0xe]
]

In [54]:
def InvMixColumns(state):
    result = [0 for x in range(len(state))]
    
    # Matrix multiplication
    for i in range (4):
        for j in range(4):
            # Initialize tmp
            tmp = 0
            
            for k in range(4):
                tmp ^= PolyMul(GetByte(state[i],k), INV_MIXCOL_MATRIX[j][k])
                
            result[i] = (result[i] << 8) | tmp

    return result

<h3>Try Yourself</h3>

In [55]:
#Initial Data State
stateInvMixColumn = [0xd42711ae, 0xe0bf98f1, 0xb8b45de5, 0x1e415230]

#Data going through MixColumn Process
stateInvMixColumn = MixColumns(stateInvMixColumn)

DisplayState(stateInvMixColumn)
#Output after MixColumn Process

65 68 14 9D 
07 C7 C9 5A 
38 7C 82 AB 
16 E5 EB 51 


In [56]:
#Data going through InvMixColumn Process
stateInvMixColumn = InvMixColumns(stateInvMixColumn)

DisplayState(stateInvMixColumn)    
#Output after Inverse MixColumn Process

D4 E0 B8 1E 
27 BF B4 41 
11 98 5D 52 
AE F1 E5 30 


<h1>Combine all AES decryption function</h1>

In [57]:
# The inverse of a normal round
def InvRound(state, roundkey):
    result = AddRoundKey(state, roundkey)
    result = InvMixColumns(result)
    result = InvShiftRows(result)
    result = InvSubBytes(result)
    
    return result

In [58]:
# The inverse of final round
def InvFinalRound(state, roundkey):
    result = AddRoundKey(state, roundkey)
    result = InvShiftRows(result)
    result = InvSubBytes(result)
    
    return result

In [59]:
def AESDecrypt(state, cipherkey):
    # Use same ExpandKey function to expand the cipher key
    expandedkey = ExpandKey(cipherkey)
    
    # Determine number of rounds based on cipher key length
    if len(cipherkey) == 4: # 128-bits key, 10 rounds
        roundnum = 10
    elif len(cipherkey) == 6: #192-bits key, 12 rounds
        roundnum = 12
    elif len(cipherkey) == 8: #256-bits key, 14 rounds
        roundnum = 14
    else:
        # Throw error here
        raise ValueError("Cipher key format incorrect!")
    
    # Inverse the Final Round
    result = InvFinalRound(state, GetRoundKey(expandedkey, roundnum))
    
    # Nr-1 rounds
    for i in range(roundnum-1 , 0, -1):
        result = InvRound(result, GetRoundKey(expandedkey, i))
    
    # Add cipher key
    result = AddRoundKey(result, cipherkey)
    
    return result

<h3>Try Yourself</h3>

In [60]:
tstate = [0x3243f6a8, 0x885a308d, 0x313198a2, 0xe0370734]
tkey = [0x2b7e1516, 0x28aed2a6, 0xabf71588, 0x09cf4f3c, 0x2b7e1516, 0x28aed2a6]

tresult = AESEncrypt(tstate, tkey)

DisplayState(tresult)

4F 57 BB DE 
CB 84 77 32 
8D A2 DB 17 
B8 C1 7E AC 


In [61]:
tstate = [0x4fcb8db8, 0x5784a2c1, 0xbb77db7e, 0xde3217ac]
tkey = [0x2b7e1516, 0x28aed2a6, 0xabf71588, 0x09cf4f3c, 0x2b7e1516, 0x28aed2a6]

tresult = AESDecrypt(tstate, tkey)

DisplayState(tresult)

32 88 31 E0 
43 5A 31 37 
F6 30 98 07 
A8 8D A2 34 


<h1>AES Test</h1>

<h3>Random Test</h3>

In [62]:
import random
import time

# Generate a random plaintext-key pair and test the result
def AESTest(keylength):
    tinput = [random.randrange(0x0, 0xffffffff) for x in range(4)]
    tkey = [random.randrange(0x0, 0xffffffff) for x in range(keylength // 32)]
    
    tencrypted = AESEncrypt(tinput, tkey)
    toutput = AESDecrypt(tencrypted, tkey)

    return (tinput == toutput)

def RandomTest():
    count1 = 0
    count2 = 0
    count3 = 0
    # Test AES128 1000 times and record time taken
    start1 = time.time()
    for i in range(1000):
        if AESTest(128):
            count1 += 1
    end1 = time.time()
    
    # Test AES192 1000 times and record time taken
    start2 = time.time()
    for i in range(1000):
        if AESTest(192):
            count2 += 1
    end2 = time.time()  
    
    # Test AES256 1000 times and record time taken
    start3 = time.time()
    for i in range(1000):
        if AESTest(256):
            count3 += 1
    end3 = time.time()
            
    print("AES128 Success:", count1)
    print("AES128 time taken: ",end1 - start1)
    print("AES192 Success:", count2)
    print("AES192 time taken: ",end2 - start2)
    print("AES256 Success:", count3)
    print("AES256 time taken: ",end3 - start3)
    
RandomTest()

AES128 Success: 1000
AES128 time taken:  8.491880416870117
AES192 Success: 1000
AES192 time taken:  7.375881910324097
AES256 Success: 1000
AES256 time taken:  14.251090049743652


In [63]:
# Converting from a string to a list of words
def StringtoState(hextext):
    # Pad hextext with 0 if length of hextext less than 32
    if len(hextext) < 32:
        pad_len = 32 - len(hextext)
        hextext = '0' * pad_len + hextext
    
    # Pad hextext with 0 if length of hextext is not a multiple of 4
    elif (len(hextext) % 8) != 0:
        pad_len = 8 - (len(hextext) % 8)
        hextext = '0' * pad_len + hextext
    
    # Initialize result list
    result = [0] * -(len(hextext) // -8)
    
    # Slice the hextext to a list of strings in group of 8
    state = [hextext[i:i+8] for i in range(0, len(hextext), 8)]
    
    # Convert from string to integer
    for i in range(len(result)):
        result[i] = int(state[i], 16)
        
    return result

In [64]:
def StatetoString(state):
    result = ""
    
    for i in range(len(state)):
        result += "{:08x}".format(state[i])
        
    return result

In [65]:
def AESEncryptStr(hexstring, cipherkey):
    # 
    result = StatetoString(AESEncrypt(StringtoState(hexstring), StringtoState(cipherkey)))
    return result

In [66]:
def AESDecryptStr(hexstring, cipherkey):
    # 
    result = StatetoString(AESDecrypt(StringtoState(hexstring), StringtoState(cipherkey)))
    return result

In [74]:
import csv
import time

def KnownAnswerTest(file):
    # Open the input csv test suite
    with open(file, newline='',) as csvfile:
        #Initialize a csv reader
        reader = csv.reader(csvfile)
        enpass = 0
        totaltest = 0

        start = time.process_time()
        for row in reader:
            # row[0]: plaintext 
            # row[1]: key
            # row[2]: ciphertext
            if (AESEncryptStr(row[0], row[1]) == row[2] and
                AESDecryptStr(row[2], row[1]) == row[0]):
                enpass += 1
            else:
                print("Test failed!")

            totaltest += 1
        
        # Print test results
        print("File path: ", file)
        print(time.process_time() - start)
        print("Total number of records tested: ",totaltest)
        print("Total number of tests passed: ",enpass, "\n")
        
KnownAnswerTest("./testsuite/GFSbox .csv")
KnownAnswerTest("./testsuite/KeySbox.csv")
KnownAnswerTest("./testsuite/VarTxt.csv")
KnownAnswerTest("./testsuite/VarKey.csv")

File path:  ./testsuite/GFSbox .csv
0.109375
Total number of records tested:  18
Total number of tests passed:  18 

File path:  ./testsuite/KeySbox.csv
0.296875
Total number of records tested:  61
Total number of tests passed:  61 

File path:  ./testsuite/VarTxt.csv
1.5
Total number of records tested:  384
Total number of tests passed:  384 

File path:  ./testsuite/VarKey.csv
2.28125
Total number of records tested:  576
Total number of tests passed:  576 



In [68]:
def TexttoHexstr(text):
    result = ""
    bytesstring = [ord(c) for c in text]
    
    for item in bytesstring:
        result += "{:02x}".format(item)
        
    if len(result) > 32:
        raise ValueError("Input too long!")
    else:
        pad_length = 32 - len(result)
        result = '0'*pad_length + result
    
    return result

In [69]:
def HexstrtoText(hexstring):
    result = ""
    # Split hexstring to a list every two characters
    listoftext = [hexstring[i:i+2] for i in range(0, len(hexstring), 2)]
    
    # Convert from hexstring to integer
    for i in range(len(listoftext)):
        charnum = int(listoftext[i], 16)
        if charnum != 0:
            result += chr(charnum)
        
    return result

In [70]:
import hashlib

def AESEncryptApplication():
    user_plaintext = TexttoHexstr(input("Please enter your input string: "))
    hashed_key = hashlib.md5(input("Please enter your key: ").encode()).hexdigest()
    
    cipher_text = AESEncryptStr(user_plaintext, hashed_key)
    print("Your secret message is: ", cipher_text)

AESEncryptApplication()

Please enter your input string: LifeIsStrange
Please enter your key: 654321
Your secret message is:  3d674569bffe0b34d99345ebf0d7cbb1


In [71]:
def AESDecryptApplication():
    user_ciphertext = input("Please enter your secret message: ")
    hashed_key = hashlib.md5(input("Please enter your key: ").encode()).hexdigest()
    
    plain_text = HexstrtoText(AESDecryptStr(user_ciphertext, hashed_key))
    print("Your decrypted message is: ",plain_text)
    
AESDecryptApplication()

Please enter your secret message: 3d674569bffe0b34d99345ebf0d7cbb1
Please enter your key: 654321
Your decrypted message is:  LifeIsStrange


<h1>Reference List</h1>
<p>
https://en.wikipedia.org/wiki/Advanced_Encryption_Standard

https://en.wikipedia.org/wiki/Finite_field_arithmetic

http://statmath.wu.ac.at/courses/data-analysis/itdtHTML/node55.html

https://sites.math.washington.edu/~morrow/336_12/papers/juan.pdf<p>

<p>Worked as part of Final Year Project by Zhang Chengxuan</p>